<div style='background-color: #e3a8b6;
    border: 0.5em solid black;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Exercice</h2>
    <h1>Découpage administratif</h1>
</div>

Grâce à l'API [Découpage administratif](https://geo.api.gouv.fr/decoupage-administratif/communes), le gouvernement français met à disposition des données sur les communes : population, superficie, limites géographiques, etc.

**(1)** ✏️ Cliquer le lien [https://geo.api.gouv.fr/communes?nom=Nogent-sur-Marne&boost=population&fields=population,surface,centre,contour](https://geo.api.gouv.fr/communes?nom=Nogent-sur-Marne&boost=population&fields=population,surface,centre,contour) puis déterminer quelles sont les données renvoyées par l'API.

**(2)** ✏️ Sachant que Nogent-sur-Marne a une superficie de 2,8 km², déterminer dans quelle unité est exprimée la surface renvoyée par l'API.

**(3)** 💻 Définir une fonction `interroger_API_decoupage_admin` qui prend en paramètre d'entrée une chaîne de caractères correspondant à un nom de ville française et qui renvoie un dictionnaire dont les clés sont `'nom'`, `'population'`, `'superficie'`, `'densité'`, `centre` et `'contour'`. La superficie sera donnée en kilomètres carrés et arrondie au centième. La densité sera donnée en habitants par kilomètre carré et arrondie à l'unité.

In [1]:
import requests

In [2]:
def interroger_API_decoupage_admin(nom):
    """
    Récupère des informations sur une commune française via l'API Découpage administratif.
    - Entrée : nom (chaîne de caractères, nom d'une commune française)
    - Sortie : dictionnaire dont les clés sont 'nom', 'population', 'superficie' (en km2), 'densité',
               'centre' (coordonnées géographiques du centre de la commune), 'contour' (au format GeoJSON)
    """
    url = f"https://geo.api.gouv.fr/communes?nom={nom}&boost=population&fields=population,surface,centre,contour"
    reponse = requests.get(url)
    reponse = reponse.json()
    nom = reponse[0]['nom']
    population = reponse[0]['population']
    superficie = reponse[0]['surface'] / 100
    coord_centre = reponse[0]['centre']['coordinates']
    contour = reponse[0]['contour']
    return {'nom': nom, 'population': population, 'superficie': round(superficie, 2),
            'densité': round(population / superficie), 'centre': (coord_centre[1], coord_centre[0]),
            'contour': contour}

In [3]:
interroger_API_decoupage_admin('Nogent-sur-Marne')

{'nom': 'Nogent-sur-Marne',
 'population': 31947,
 'superficie': 2.8,
 'densité': 11408,
 'centre': (48.8363, 2.4788),
 'contour': {'type': 'Polygon',
  'coordinates': [[[2.467232, 48.839094],
    [2.46731, 48.839111],
    [2.469093, 48.839835],
    [2.469518, 48.840014],
    [2.468691, 48.840898],
    [2.472246, 48.842054],
    [2.472604, 48.840631],
    [2.47272, 48.840472],
    [2.473151, 48.840052],
    [2.474219, 48.840477],
    [2.476033, 48.841145],
    [2.476357, 48.841942],
    [2.476777, 48.842543],
    [2.476802, 48.842744],
    [2.477596, 48.842912],
    [2.478066, 48.843101],
    [2.478087, 48.843077],
    [2.479223, 48.843493],
    [2.479883, 48.841895],
    [2.482163, 48.842389],
    [2.483192, 48.84236],
    [2.484117, 48.843097],
    [2.485494, 48.843186],
    [2.485813, 48.843394],
    [2.48557, 48.844276],
    [2.486504, 48.84452],
    [2.486481, 48.845093],
    [2.486539, 48.845338],
    [2.486627, 48.845543],
    [2.487296, 48.845626],
    [2.487513, 48.845773],
  

⚠️ La suite de l'exercice utilise le module `folium`, qui peut être installé en exécutant la cellule ci-dessous.

In [ ]:
import sys
!{sys.executable} -m pip install folium

On définit maintenant une carte grâce au module `folium`.

In [4]:
import folium

In [5]:
ma_carte = folium.Map(location=(48.853,2.349), tiles='OpenStreetMap', zoom_start=11)

Cette carte représente Paris et une partie de sa banlieue. Elle peut être affichée dans le carnet _Jupyter_ en exécutant la cellule suivante.

In [6]:
ma_carte

**(4)** 💻 Définir une procédure `dessiner_commune` qui prend en paramètre d'entrée une chaîne de caractères correspondant à un nom de ville française, qui dessine les contours de la commune et qui place un marqueur au centre de la commune. En cas de survol du marqueur par la souris, le nom de la ville doit s'afficher, et en cas de clic sur le marqueur, une fenêtre popup doit s'ouvrir, en indiquant la population et la superficie de la commune.

Vous pourrez vous aider de la [documentation](http://python-visualization.github.io/folium/modules.html#module-folium.map) du module `folium`, en particulier les explications sur `Marker`,`Popup` et `GeoJson`.

In [7]:
def dessiner_commune(nom, carte):
    """
    Ajoute sur une carte la représentation d'une commune.
    - Entrées : nom (chaîne de caractères, nom d'une commune française), carte (carte créée avec le module folium)
    Effet de bord : modification de la carte
    """
    dico = interroger_API_decoupage_admin(nom)
    html = f"Population : {dico['population']} hab.<br>Superficie : {dico['superficie']} km2"
    folium.Marker(dico['centre'], tooltip=dico['nom'], popup=folium.Popup(html=html, max_width=300)).add_to(carte)
    folium.GeoJson(dico['contour']).add_to(carte)

In [8]:
ma_carte = folium.Map(location=(48.853,2.349), tiles='OpenStreetMap', zoom_start=11)
dessiner_commune('Nogent-sur-Marne', ma_carte)
dessiner_commune('Argenteuil', ma_carte)
dessiner_commune('Meudon', ma_carte)
dessiner_commune('Saint-Denis', ma_carte)
dessiner_commune('Clermont-Ferrand', ma_carte)
ma_carte # affichage de la carte sous la cellule

Lorsque plusieurs villes portent le même nom, ce sont les caractéristiques de la ville la plus peuplée qui sont renvoyées par la fonction `interroger_API_decoupage_admin`. En effet, dans la définition de cette fonction, l'API est interrogée avec l'argument _boost=population_, ce qui met en avant (en première position dans `reponse`) la ville la plus peuplée. C'est la raison pour laquelle l'appel `dessiner_commune('Saint-Denis')` marque la ville de Saint-Denis de la Réunion, qui est plus peuplée que Saint-Denis en Seine-Saint-Denis.